# Cauchy Method

$f(x_k+td_k) \approx f(x_k)+t\nabla f(x_k)'d_k+\frac{1}{2}t²d_k'\nabla²f(x_k)d_k$

$t=\large -\frac{\nabla f(x_k)'d_k}{d_k'\nabla²f(x_k)d_k}$

$d_k=-\nabla f(x_k)$

In [2]:
import numpy as np 

class f:
    def __init__(self, list_of_parameters, initial_inputs):
        self.params = np.array(list_of_parameters)
        self.inputs = np.array(initial_inputs)
    
    def update_inputs(self, list_of_inputs):
        self.inputs = np.array(list_of_inputs)
        return self
        
    def evaluate(self):
        return self.params.dot(self.inputs ** 2)
    
    def gradient(self):
        return 2 * self.inputs * self.params
        
    def hessian(self):
        dim = len(self.params)
        hess = np.zeros((dim,dim))
        np.fill_diagonal(hess, 2 * self.params)
        return hess

In [3]:
func = f([1,2],[1,2])

In [4]:
def cauchy(params, init_inputs):
    k=0
    func = f(params,init_inputs)
    x = init_inputs
    
    while np.linalg.norm(func.gradient()) > 0.0001 and k<10001:
        grad = func.gradient()
        num = grad.T.dot(grad)
        denom = grad.T.dot(func.hessian()).dot(grad)
        t = num/denom
        x_new = x - t*grad
        func.update_inputs(x_new)
        x = x_new.copy()
        k=k+1
        
    print('The function converged at iteration {}'.format(k))
    print('And f has the value of {}'.format(func.evaluate()))
        
    return(x,k)

In [5]:
cauchy([1,2],[1,2])

The function converged at iteration 7
And f has the value of 9.259281190329347e-11


(array([ 9.47559862e-06, -1.18444983e-06]), 7)

In [6]:
def gradient_descent_backtracking_cauchy(f_params, initial_inputs, t0):
    k=0
    x = initial_inputs
    func = f(f_params, x)
    list_t = []
    
    while (np.linalg.norm(func.gradient()) > 0.0001 and k < 10001):
        dk = func.gradient()
        t_cauchy = dk.T.dot(dk)/(dk.T.dot(func.hessian()).dot(dk))
        t = max(t0, t_cauchy)
        #print(t)
        list_t.append(t)
        
        x_next = x - t*dk
        func.update_inputs(x_next)
        x = x_next.copy()
        k = k + 1
    
    print('The process converged in the interation: {}'.format(k))
    print('The final value of f was: {}'.format(func.evaluate()))

    return(x,k,list_t)

In [7]:
gradient_descent_backtracking_cauchy([1,2],[1,2],0.05)[0]

The process converged in the interation: 7
The final value of f was: 9.259281190329347e-11


array([ 9.47559862e-06, -1.18444983e-06])

In [8]:
params = [1,1,1,1,1,10,10,10,10,10]
init = [1,1,1,1,1,1,1,1,1,1]

In [9]:
cauchy(params, init)

The function converged at iteration 10
And f has the value of 2.552379523939185e-10


(array([2.15422609e-06, 2.15422609e-06, 2.15422609e-06, 2.15422609e-06,
        2.15422609e-06, 2.15422609e-06, 2.15422609e-06, 2.15422609e-06,
        2.15422609e-06, 2.15422609e-06]), 10)

In [10]:
gradient_descent_backtracking_cauchy(params, init, 0.05)[0]

The process converged in the interation: 10
The final value of f was: 2.552379523939185e-10


array([2.15422609e-06, 2.15422609e-06, 2.15422609e-06, 2.15422609e-06,
       2.15422609e-06, 2.15422609e-06, 2.15422609e-06, 2.15422609e-06,
       2.15422609e-06, 2.15422609e-06])

# Conjugate gradients (CG)

$min \frac{1}{2} x'Ax - b'x \rightarrow Ax-b=0$

In [11]:
class quadratic:
    
    def __init__(self, A, b, initial_input):
        self.A = np.array(A)
        self.b = np.array(b)
        self.inputs = np.array(initial_input)
    
    def update_inputs(self, list_of_inputs):
        self.inputs = np.array(list_of_inputs)
        return self
    
    def evaluate(self):
        x = self.inputs
        return 0.5*x.T.dot(self.A).dot(x) - self.b.dot(x)
    
    def gradient(self):
        return self.A.dot(self.inputs)-self.b
    
    def hessian(self):
        return self.A

In [12]:
A = np.array([[3,2],[2,6]])
b = np.array([2,-8])

In [13]:
q=quadratic(A,b,np.array([1,1]))

In [14]:
q.evaluate()

12.5

In [15]:
q.gradient()

array([ 3, 16])

In [16]:
q.hessian()

array([[3, 2],
       [2, 6]])

In [17]:
def CG_gram_schmidt(A, b, initial_input):
    
    q = quadratic(A,b,initial_input)
    g = q.gradient()
    k = 0
    d_list = []
    x = initial_input
    
    while np.linalg.norm(g) > 0.0000001 and k < 1001:
        
        dk = -g
        
        for d in d_list:
            dk += (g.T.dot(A).dot(d))/(d.T.dot(A).dot(d))*d
        
        ak = -(g.T.dot(dk))/(dk.T.dot(A).dot(dk))
        x_new = x + ak*dk
        
        q.update_inputs(x_new)
        g = q.gradient()
        
        k += 1
        x = x_new.copy()
        d_list.append(dk)
        
        if k%100==0:
            print(q.evaluate(), np.linalg.norm(g))
        
    print("The optimisation converged in the {}".format(k)+" iteration.")
    return(x,k,q.evaluate(),d_list)

In [18]:
CG_gram_schmidt(A,b,np.array([1,2]))[0]

The optimisation converged in the 2 iteration.


array([ 2., -2.])